In [2]:
import sys
import os

import pynq

import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual, HBox
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..')))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

!python --version

print(sys.version)

from RFSoC_Daq import RFSoC_Daq
from Biquad import Biquad
from Biquad_Daq import Biquad_Daq
from SimBiquad import SimBiquad

from Utils.PlotTools import PlottingTools as pt

from Waveforms.Waveform import Waveform
from Waveforms.Gated import Gated
from Waveforms.Filterred import Filterred

Python 3.10.4
3.10.4 (main, Apr  2 2022, 09:04:19) [GCC 11.2.0]


TypeError: metaclass conflict: the metaclass of a derived class must be a (non-strict) subclass of the metaclasses of all its bases

In [ ]:
daq = RFSoC_Daq(None, None, 4, 2**10)

In [ ]:
def update_plot(A, B, P, theta):
    daq.reset()
    
    daq.setA(A)
    daq.setB(B)
    daq.setP(P)
    daq.setTheta(np.pi*theta)
    
    daq.run_Biquad()
    daq.JupyterAcquire()
    
    daq.waveforms[0].setRMS()
    amp = daq.waveforms[0].RMS
    
    biquad = SimBiquad(data=daq.adcBuffer[0] >> 4, A=A, B=B, P=P, theta=np.pi*theta)  
    biquad.set_daq_coeffs(daq.get_coeffs())
    
    biquad.single_zero_fir()     #.get_u()
    biquad.first_constants()    #.get_decimated1()
    biquad.second_constants()    #.get_decimated2()
    biquad.IIR_calculation()   #get_decimated
    biquad.incremental() #get_biquad()
        
    sim_output = biquad.get_biquad()
    
    fig, ((axB,axS,axR),(ax0, ax1, ax2)) = plt.subplots(2, 3, figsize=(35, 15))
        
    daq.waveforms[2].plotWaveform(axB, title="Biquaded ADC224_T0_CH0")
    daq.waveforms[2].plotFFT(ax0, title="Biquaded FFT")
    
    sim_output.plotWaveform(axS, title="Simulated Biquad")
    sim_output.plotFFT(ax1, title="Simulated FFT")
    
    daq.waveforms[0].plotWaveform(axR, title="Raw ADC224_T0_CH0")
    daq.waveforms[0].plotFFT(ax2, title="Raw FFT")
    
    plt.show()
    
    
A_slider = widgets.FloatSlider(min=-8, max=7.9, step=0.05, value=0.91187514, description='A:', continuous_update=False)
B_slider = widgets.FloatSlider(min=-8, max=7.9, step=0.05, value=-1.04083903, description='B:', continuous_update=False)
P_slider = widgets.FloatSlider(min=0, max=1, step=0.01, value=0.9076069015316293, description='P:', continuous_update=False)
Theta_slider = widgets.FloatSlider(min=0, max=2, step=0.01, value=0.9601495285661642/np.pi, description='Theta:', continuous_update=False)

# Create interactive plot
plot_output = widgets.Output()
    
def update(change):
    with plot_output:
        plot_output.clear_output(wait=True)
        update_plot(A_slider.value, B_slider.value, P_slider.value, Theta_slider.value)
    
A_slider.observe(update, names='value')
B_slider.observe(update, names='value')
P_slider.observe(update, names='value')
Theta_slider.observe(update, names='value')

# Layout
slider_box = widgets.HBox([A_slider, B_slider, P_slider, Theta_slider])
ui = widgets.VBox([slider_box, plot_output])

# Display
display(ui)

# Initial plot
update(None)